In [143]:
import re
import docx
import csv
from os import listdir

<h4>subtitles from transcript</h4>

In [144]:

def convert_to_srt(input_file, output_file):
    doc = docx.Document(input_file)
    lines = []
    for paragraph in doc.paragraphs:
        lines.append(paragraph.text)

    subtitles = []
    subtitle_block = ""
    for line in lines:
        line = line.replace('*', '')  # Remove asterisks from the line
        line = line.replace('–', '-')  # Replace different dash characters with a standard dash
        if line.strip() == "":
            if subtitle_block:
                subtitles.append(subtitle_block)
                subtitle_block = ""
        else:
            subtitle_block += line +"\n"
        
    with open(output_file, 'w') as f:
        idx = 1
        for subtitle in subtitles:
            parts = subtitle.split("\n", 1)
            time_range_match = re.search(r'(\d+:\d+\s*-\s*\d+:\d+)', parts[0])
            if time_range_match:
                time_range = time_range_match.group(1)
                text = parts[1].strip()
                
                start_time, end_time = map(str.strip, time_range.split("-"))
                start_time_formatted = start_time + ',000'
                end_time_formatted = end_time + ',000'
                srt_entry = f"{idx}\n{start_time_formatted} --> {end_time_formatted}\n{text}\n\n"
                f.write(srt_entry)
                
                idx += 1

### test (dunas)
# en_input_file = "../02_Transcripts/Wayuu/Transcripts (EN)/dunas (EN).docx"
# en_output_file = "../05_Subtitles/Wayuu/SRT Exports (EN)/dunas (EN).srt"
# convert_to_srt(en_input_file, en_output_file)

# es_input_file = "../02_Transcripts/Wayuu/Transcripts (ES)/dunas (ES).docx"
# es_output_file = "../05_Subtitles/Wayuu/SRT Exports (ES)/dunas (ES).srt"
# convert_to_srt(es_input_file, es_output_file)

# print("Conversion complete!")


In [145]:
def txtToSrtPipeline(source, destination):
    inputFiles=listdir(source) 
    for inputFile in inputFiles:
        if "(en)" in inputFile.lower() or "(es)" in inputFile.lower():
            fileName=inputFile.split("/")[-1].split(".")[0]
            outputFile=destination+fileName.lower()+".srt"
            inputFile=source+inputFile
            convert_to_srt(inputFile, outputFile)
            print(fileName,"conversion complete!\n")
        
### test (wayuu)
# source="../02_Transcripts/Wayuu/Transcripts (EN)/"
# destination="../05_Subtitles/Wayuu/SRT Exports (EN)/"

# txtToSrtPipeline(source, destination)

# source="../02_Transcripts/Wayuu/Transcripts (ES)/"
# destination="../05_Subtitles/Wayuu/SRT Exports (ES)/"

# txtToSrtPipeline(source, destination)

<h3> Convert subtitles to CSV </h3>

In [146]:
import re
import csv

def srt_to_csv(en_srt_file, es_srt_file, output_csv_file):
    subtitles_dict = {}  # Dictionary {time_range: (start_time, end_time, en_text, es_text)}

    # Read English subtitles
    with open(en_srt_file, 'r') as f_en:
        subtitle_lines_en = f_en.read().split('\n\n')

        for subtitle in subtitle_lines_en:
            lines = subtitle.strip().split('\n')
            if len(lines) >= 3:
                time_range = lines[1]
                text = '\n'.join(lines[2:])

                # Extract start and end times
                time_parts = time_range.split(" --> ")
                start_time, end_time = time_parts[0], time_parts[1]

                if time_range in subtitles_dict:
                    subtitles_dict[time_range] = (start_time, end_time, text, subtitles_dict[time_range][3])
                else:
                    subtitles_dict[time_range] = (start_time, end_time, text, '')

    # Read Spanish subtitles
    with open(es_srt_file, 'r') as f_es:
        subtitle_lines_es = f_es.read().split('\n\n')

        for subtitle in subtitle_lines_es:
            lines = subtitle.strip().split('\n')
            if len(lines) >= 3:
                time_range = lines[1]
                text = '\n'.join(lines[2:])

                # Extract start and end times
                time_parts = time_range.split(" --> ")
                start_time, end_time = time_parts[0], time_parts[1]

                if time_range in subtitles_dict:
                    subtitles_dict[time_range] = (subtitles_dict[time_range][0], subtitles_dict[time_range][1], subtitles_dict[time_range][2], text)
                else:
                    subtitles_dict[time_range] = (start_time, end_time, '', text)

    # Write combined subtitles to a CSV file
    with open(output_csv_file, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Start Time', 'End Time', 'English', 'Spanish','Tag','Comment','Highlight'])  # Write header

        for _, (start_time, end_time, en_text, es_text) in sorted(subtitles_dict.items()):
            csv_writer.writerow([start_time, end_time, en_text, es_text])

# Test
en_srt_file = "../05_Subtitles/Wayuu/SRT Exports (EN)/dunas (EN).srt"
es_srt_file = "../05_Subtitles/Wayuu/SRT Exports (ES)/dunas (ES).srt"
output_csv_file = "../subtitles_combined.csv"  # Replace with your desired output CSV file

srt_to_csv(en_srt_file, es_srt_file, output_csv_file)
print("Conversion to CSV complete!")


IndexError: list index out of range

<h4>subtitles from script</h4>


In [ ]:
#check/fix

# def convertTimecodeToSubFormat(timecode):
#     timecodeSplit=timecode.split(":")
#     timecodeSplit[3]=str(int(int(timecodeSplit[3])/24*1000))
#     timecodeNew = ":".join(timecodeSplit[:3])+","+timecodeSplit[3]
#     return timecodeNew

# #convertTimecodeToSubFormat("01:02:03:04")   #test

# # the generate_subtitles() function reads a script CSV file, performs time calculations
# # and formatting, and writes the generated subtitles to a final subtitle file in the SRT format. if no time is found
# # it adds 5 seconds to the last time.

# def generate_subtitles(start_time, end_time, text_line):
#     global subtitle_counter , final_subtitle_file

#     with open(final_subtitle_file, 'a') as srt_file:


#         srt_file.write(str(subtitle_counter) + '\n')

#         srt_file.write(convertTimecodeToSubFormat(start_time) +' --> ' + convertTimecodeToSubFormat(end_time) + '\n')

#         srt_file.write(clean_text(text_line) + '\n\n')  # Cleaned text
#         subtitle_counter += 1

<h3> Convert Script to coded CSV </h3>

In [ ]:

# The match_script_to_csv() function performs a matching operation between a script file and a
#  folder containing data from multiple CSV files
# in order to generate a csv version of the script with links to files and timecodes

def match_script_to_csv(script_path):

    combined_df = pd.DataFrame()

    for filename in os.listdir(csv_folder_path):
        if filename.endswith("csv"):
            file_path = os.path.join(csv_folder_path, filename)
            df = pd.read_csv(file_path)
            df["NARRATOR"]=(filename.split(" ")[1])
            df['FILEPATH'] = file_path
            combined_df = pd.concat([combined_df, df], ignore_index=True)

    matched_rows = []

    with open(script_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    narrator = None
    for line in lines:
        line=line.strip()
        if line=="":
            pass
        elif line[:3]=="###":
            narrator=line.strip()[3:]
        else:
            matched_rows_df = match_line(line, narrator, combined_df)
            cleaned_line = clean_text(line)
            if not matched_rows_df.empty:
                for index,row in matched_rows_df.iterrows():
                    print(row)
                    time_range = [s.strip() for s in row['TIME'].split("-")]
                    start_time = timecode_to_frames(time_range[0])
                    end_time = timecode_to_frames(time_range[1])
                    total_duration = end_time - start_time

                    text = clean_text(str(row['TEXT']).lower())

                    ##if the row matches completely, the new and original start/end times should match

                    print(text,cleaned_line)

                    start_index, end_index = get_character_indices(text, cleaned_line)
                    total_characters = len(list(text))

                    print(start_time,total_duration,total_characters,start_index)

                    new_start_time = start_time + total_duration/total_characters*start_index
                    new_end_time = start_time + total_duration/total_characters*end_index

                    new_row = {
                        'Text': line,
                        'Narrator': narrator,
                        'Timecode Range': frames_to_timecode(new_start_time) +" - " + frames_to_timecode(new_end_time),
                        'FilePath': row['FILEPATH']
                        }
            else:
                ##if no match is found, the line is added anyway but with no timecode or filepath
                new_row = {
                        'Text': line,
                        'Narrator': narrator,
                        'Timecode Range': None,
                        'FilePath': None
                    }
            matched_rows.append(new_row)

    matched_df = pd.DataFrame.from_records(matched_rows)
    return matched_df

# match_script_to_csv(script_path).to_csv(script_csv_output_path) #test